# Load point values

This is a demo that queries a list of points and orders by risk according to Wildfire Risk based on the [Historical Wildfire](https://data.spatiafi.com/dataset/spatiafi-wildfire-risk-static-global-v1.0) dataset.  We start by loading our App Credentials, and then using the [point](https://docs.spatiafi.com/api/#operation/point_api_point__lon___lat__get) endpoint to fetch fire risk values at the points of interest.

---

Install `spatiafi` (if not already installed):

In [ ]:
!sudo apt -y install libgeos-dev

In [ ]:
%pip install --upgrade spatiafi cartopy xarray

In [ ]:
from datetime import datetime, timedelta

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spatiafi
import xarray as xr

In [ ]:
session = spatiafi.get_session()

In [ ]:
session

## Point Value
This section takes a list of example asset locations (lats/lons) and queries the historical fire risk at each location. This should give us an understanding of the medium-term fire risk for our assets.

In [ ]:
asset_coords = [
    ["-104.18", "55.48"],
    ["-92.88", "55.74"],
    ["-113.46", "56.99"],
    ["-112.11", "46.31"],
    ["5.55", "6.89"],
    ["-89.44", "31.01"],
    ["137.41", "-24.57"],
]

values = np.zeros(len(asset_coords))

for i in range(len(asset_coords)):
    url = (
        "https://api.spatiafi.com/api/point/"
        + asset_coords[i][0]
        + ","
        + asset_coords[i][1]
    )

    query = {"item_id": "ce-wildfire-risk-static-global-v1.0", "bidx": "1"}

    response = session.get(url, params=query)

    data = response.json()
    values[i] = data["values"][0]

    print(data)

We now plot these values so that we can more clearly see which assets are of greatest risk.

In [ ]:
# Convert asset coordinates to float
lon_float = [float(coord[0]) for coord in asset_coords]
lat_float = [float(coord[1]) for coord in asset_coords]

data = xr.DataArray(values, dims=("points",), coords={"points": range(len(values))})

# Add latitude and longitude as coordinates
data = data.assign_coords(
    latitude=("points", lat_float), longitude=("points", lon_float)
)

fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot the data using scatter plot
plt.scatter(
    data["longitude"],
    data["latitude"],
    c=data,
    cmap="Reds",
    transform=ccrs.PlateCarree(),
)

# Add coastlines
ax.coastlines()

# Show the plot
plt.show()

We can see that our assets with the largest historical fire risk are largely located in Canada, consistent with the large wildfires over the past few summers.

In [ ]:
df = pd.DataFrame({"Lon": lon_float, "Lat": lat_float, "Fire Risk": values})

df.sort_values("Fire Risk", ascending=False)

We can also easily save the data out as a pandas DataFrame, allowing us to do things like sort by Fire Risk to see the assets at greatest risk.

## Point Values Over Time
We now include an example that queries timeseries from the a point in Northern Canada. This function can be used to, for example, track trends in the Fire Weather Index (FWI) to see if there will be assets we should be concerned about in a few days' time. Here we use it to query the next 7 days of data at each location and calculate the trend to see if there is a notable increase or decrease.

In [ ]:
now = datetime.now()
in_seven_days = now + timedelta(days=6)

start_date = now.date().isoformat()
end_date = in_seven_days.date().isoformat()
date_range = f"{start_date}/{end_date}"

url = (
    "https://api.spatiafi.com/api/timeseries/point/"
    + asset_coords[0][0]
    + ","
    + asset_coords[0][1]
)

query = {
    "coll_id": "climate-engine-cfsv2-era5-land-fire-weather-index-forecasts",
    "datetime": date_range,
    "bidx": "1",
}

response = session.get(url, params=query)

data = response.json()

# Convert dictionary to a DataFrame
df = pd.DataFrame(data.values(), index=pd.to_datetime(list(data.keys())))
df.index.name = "Date"

# Convert values from lists to integers
df["values"] = df["values"].apply(lambda x: float(x[0]))

df

In [ ]:
# Plot the 'values' column as a time series
plt.figure(figsize=(10, 6))
plt.plot(df.index, df["values"], marker="o", linestyle="-", color="b")
plt.xlabel("Date")
plt.ylabel("FWI")
plt.title("Time Series Plot of FWI Forecast in Northern Canada")
plt.grid(True)
plt.tight_layout()
plt.show()